In [1]:
# 修改预训练模型的缓存目录
import os

os.environ["MODELSCOPE_CACHE"] = r"G:\code\pretrain_model_dir\_modelscope"

In [ ]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('qwen/Qwen2-VL-7B-Instruct')

In [ ]:
print(model_dir)

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from modelscope import snapshot_download
import torch
model_dir = r"G:\code\pretrain_model_dir\_modelscope\qwen\Qwen2-VL-7B-Instruct"

# default: Load the model on the available device(s)
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     model_dir, torch_dtype="auto", device_map="auto"
# )

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)

In [ ]:
print(model.device, model.dtype)

In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from modelscope import snapshot_download
import torch
model_dir = r"G:\code\pretrain_model_dir\_modelscope\qwen\Qwen2-VL-7B-Instruct"

# default processer
processor = AutoProcessor.from_pretrained(model_dir)

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained(model_dir, min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")


c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\tech\Anaconda3\envs\nlp\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-10-27 11:04:12,159 - modelscope - INFO - PyTorch version 2.1.0+cu121 Found.
2024-10-27 11:04:12,163 - modelscope - INFO - TensorFlow version 2.8.0 Found.
2024-10-27 11:04:12,163 - modelscope - INFO - Loading ast index from G:\code\pretrain_model_dir\_modelscope\ast_indexer
2024-10-27 11:04:12,267 - modelscope - INFO - Loading done! 

In [2]:
print(inputs.keys())

for key, val in inputs.items():
    print(key, val.shape)
print(inputs)

dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])
input_ids torch.Size([1, 1272])
attention_mask torch.Size([1, 1272])
pixel_values torch.Size([4988, 1176])
image_grid_thw torch.Size([1, 3])
{'input_ids': tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'), 'pixel_values': tensor([[ 0.8501,  0.8647,  0.8647,  ...,  1.4207,  1.4349,  1.4349],
        [ 1.0106,  1.0106,  1.0252,  ...,  1.5060,  1.5060,  1.5202],
        [ 0.9960,  1.0106,  1.0252,  ...,  1.5060,  1.5202,  1.5202],
        ...,
        [-0.2448, -0.2594, -0.2886,  ..., -0.1862, -0.1720, -0.1862],
        [-0.1426, -0.1134, -0.0550,  ..., -0.1293, -0.1435, -0.1720],
        [-0.2448, -0.2448, -0.2594,  ..., -0.3853, -0.4279, -0.4564]],
       device='cuda:0'), 'image_grid_thw': tensor([[ 1, 58, 86]], device='cuda:0', dtype=torch.int32)}


In [ ]:
# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)